In [1]:
import pandas as pd
import numpy as np

import json

In [5]:
# Converts JSON to JSONReader object which can be used as an iterator
chunksize = 5e5
df = pd.read_json('massive_comment_dump', lines=True,chunksize=chunksize)

In [6]:
# Converts object to a list of dataframes which can be later parsed / accessed
# Need to do this with only up to 5 dataframes at a time to avoid out of memory errors 
subreddit1 = 'LateStageCapitalism' 
subreddit2 = 'Libertarian'
subreddit3 = 'Anarchism'
subreddit4 = 'politics'
subreddit5 = 'Conservative'

sub_df1 = pd.DataFrame([])
sub_df2 = pd.DataFrame([])
sub_df3 = pd.DataFrame([])
sub_df4 = pd.DataFrame([])
sub_df5 = pd.DataFrame([])

for i in df:
    sub_df1 = pd.concat([sub_df1,i[i['subreddit']==subreddit1]]) 
    sub_df2 = pd.concat([sub_df2,i[i['subreddit']==subreddit2]])
    sub_df3 = pd.concat([sub_df3,i[i['subreddit']==subreddit3]])
    sub_df4 = pd.concat([sub_df4,i[i['subreddit']==subreddit4]])
    sub_df5 = pd.concat([sub_df5,i[i['subreddit']==subreddit5]])

print(len(sub_df1))
print(len(sub_df2))
print(len(sub_df3))
print(len(sub_df4))
print(len(sub_df5))

60336
94713
12855
1833170
50545


In [55]:
# Update subreddit of interest to manuall inspect comments and preprocess
sub_df = sub_df5.copy()
subreddit = subreddit5

In [56]:
sub_df.head().transpose()

,6316,7689,12507,14380,19398
author,johnmrson,paper_boy_1,deepfeels96,deepfeels96,Belchie
author_cakeday,NaN,NaN,NaN,NaN,NaN
author_flair_css_class,None,None,None,None,None
author_flair_text,None,None,Catholic Conservative,Catholic Conservative,None
body,"According to California law, HIV is no longer ...","Funny , he was not interviewed by the FBI unti...",Nice of him to not only abandon police funeral...,"If they live in the country, they should be on...",Nothing but a flimsy excuse. There's no way th...
can_gild,True,True,True,True,True
controversiality,0,0,0,0,0
created_utc,1514764943,1514764979,1514765110,1514765158,1514765290
distinguished,None,None,None,None,None
edited,0,0,0,0,0


In [57]:
# Select only columns used in analysis
output = sub_df[['author','body','score','created_utc','id','link_id','parent_id']].copy()

In [58]:
# Remove deleted comments
output = output[output['body']!='[deleted]']
len(output)

48568

In [59]:
# Convert timestamp
conv_time = pd.to_datetime(output['created_utc'], unit='s')

output['hour'] = conv_time.dt.hour
output['day'] = conv_time.dt.day
output['month'] = conv_time.dt.month
output['year'] = conv_time.dt.year

In [60]:
print('Subreddit:',subreddit,'output')
print('Number of comments:',len(output))
print()

print(output.dtypes)
print()

for i in output.columns[2:4]:
    print(i,":",np.all(np.isfinite(output[i])))
print()
    
print(output.isnull().sum())

Subreddit: Conservative output
Number of comments: 48568

author         object
body           object
score           int64
created_utc     int64
id             object
link_id        object
parent_id      object
hour            int64
day             int64
month           int64
year            int64
dtype: object

score : True
created_utc : True

author         0
body           0
score          0
created_utc    0
id             0
link_id        0
parent_id      0
hour           0
day            0
month          0
year           0
dtype: int64


In [61]:
# Remove duplicate columns 
output = output.drop_duplicates(subset="body")

In [62]:
# Export comments in CSV
comm_title = subreddit+'_comments.csv'
output.to_csv(comm_title,index=False)

In [52]:
"""comm_title = subreddit+'_comments.JSON'
output.to_json(comm_title,orient='records', lines=True)"""